In [7]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.svm import SVC

In [9]:
options = ['blingfire','reweightedblingfire','nltk','reweightednltk','spacysm','reweightedspacysm','spacylg','reweightedspacylg','spacytrf','reweightedspacytrf']
symps = ["Anxious_Mood","Autonomic_symptoms","Cardiovascular_symptoms","Catatonic_behavior","Decreased_energy_tiredness_fatigue","Depressed_Mood","Gastrointestinal_symptoms","Genitourinary_symptoms","Hyperactivity_agitation","Impulsivity","Inattention","Indecisiveness","Respiratory_symptoms","Suicidal_ideas","Worthlessness_and_guilty","avoidance_of_stimuli","compensatory_behaviors_to_prevent_weight_gain","compulsions","diminished_emotional_expression","do_things_easily_get_painful_consequences","drastical_shift_in_mood_and_energy","fear_about_social_situations","fear_of_gaining_weight","fears_of_being_negatively_evaluated","flight_of_ideas","intrusion_symptoms","loss_of_interest_or_motivation","more_talktive","obsession","panic_fear","pessimism","poor_memory","sleep_disturbance","somatic_muscle","somatic_symptoms_others","somatic_symptoms_sensory","weight_and_appetite_change","Anger_Irritability"]

for name in options:
    df = pd.read_json(f'../data/vectorData/{name}Vectors.json', orient='records', lines=True)

    (df['gender'] == 'm') & (df['split'] == 'train')

    X_train = df.loc[(df['split'] == 'train'), symps]
    X_test = df.loc[(df['split'] == 'test'), symps]
    X_testm = df.loc[(df['gender'] == 'm') & (df['split'] == 'test'), symps]
    X_testf = df.loc[(df['gender'] == 'f') & (df['split'] == 'test'), symps]

    y_train = df.loc[(df['split'] == 'train'), ['label']]
    # y_test = df.loc[(df['split'] == 'test'), ['label']]
    # y_testm = df.loc[(df['gender'] == 'm') & (df['split'] == 'test'), ['label']]
    # y_testf = df.loc[(df['gender'] == 'f') & (df['split'] == 'test'), ['label']]

    scaler = StandardScaler()

    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)
    X_testm = scaler.transform(X_testm)
    X_testf = scaler.transform(X_testf)

    y_train = y_train.values.ravel()

    param_grid = {'C': np.logspace(-3, 3, 7),  
                'gamma': np.logspace(-3, 3, 7), 
                'kernel': ['rbf']}  

    cv = 5
    scoring = 'accuracy'

    grid = GridSearchCV(SVC(random_state=99, probability=True, class_weight='balanced'), param_grid, scoring=scoring, cv=cv, n_jobs=-1)
    grid.fit(X_train,y_train)

    y_pred = grid.predict(X_test)
    y_prob = grid.predict_proba(X_test)[:,1]

    testSet = df.loc[(df['split'] == 'test')]

    testSet['prediction'] = y_pred
    testSet['probability'] = y_prob

    testSet.to_json(f'../data/predictionData/{name}Pred.json',orient='records',lines=True)

C:\Users\bramb\AppData\Local\Temp\ipykernel_42168\643238100.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testSet['prediction'] = y_pred
C:\Users\bramb\AppData\Local\Temp\ipykernel_42168\643238100.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testSet['probability'] = y_prob
C:\Users\bramb\AppData\Local\Temp\ipykernel_42168\643238100.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the